In [1]:
%matplotlib widget

In [2]:
from aiidalab_sssp.inspect.subwidgets.periodic_table import PeriodicTable
from aiidalab_sssp.inspect.subwidgets.select import PseudoSelectWidget
from aiidalab_sssp.inspect import DB_FOLDER
import ipywidgets as ipw
from pathlib import Path
from IPython.display import display
import matplotlib.pyplot as plt

plt.ioff()

In [3]:
# If cached DB folder not exist (first time app run), create folder
DB_FOLDER.mkdir(parents=True, exist_ok=True)
ptable = PeriodicTable(cache_folder=str(DB_FOLDER))

display(ptable)

PeriodicTable(children=(PTableWidget(allElements=['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', …

In [4]:
pseudo_select = PseudoSelectWidget()

# ipw.dlink(
#     (periodic_table, 'selected_element'), 
#     (pseudo_select_widget, 'selected_element'),
# )
pseudo_select.element = 'O'

display(pseudo_select)

PseudoSelectWidget(children=(HTML(value='Please choose pseudopotentials of element O to inspect:'), SelectMult…

In [7]:
from aiidalab_sssp.inspect.subwidgets.delta import EosWidget

eos_preview = EosWidget()
ipw.dlink((pseudo_select, 'selected_pseudos'), (eos_preview, 'pseudos'))

display(eos_preview)

{'O.us.z_6.uspp.gbrv.v1.2': {'_metadata': {'_aiida_hash': 'd5fc448732d8170ec4f231d1eecc7685843cd873e8d685ab284d13332b3bdbd7',
   'ctime': '2022-05-29 14:07:52.130236+00:00',
   'uuid': 'e7caee8a-9a4c-4551-bb61-d6bf3bb14ea2'},
  'accuracy': {'bands': {'_metadata': {'_aiida_hash': 'e6c99c389e2ba1dff954abf89c80a3f2cb786fef094f47493711158b34f5eabd',
     'ctime': '2022-05-29 14:08:06.104738+00:00',
     'uuid': 'e5808405-4d54-4ff7-abd8-bef88583c930'},
    'band_structure': 'band_structure/O/O.us.z_6.uspp.gbrv.v1.2.json',
    'bands': 'bands/O/O.us.z_6.uspp.gbrv.v1.2.json'},
   'delta': {'BCC': {'ecutrho': 3600,
     'ecutwfc': 200,
     'eos': {'output_birch_murnaghan_fit': {'bulk_deriv0': 4.7404153,
       'bulk_modulus0': 0.942593,
       'bulk_modulus0_GPa': 151.0201149,
       'bulk_modulus0_GPa_unit': 'GPa',
       'bulk_modulus0_unit': 'eV A^3',
       'energy0': -434.9586341,
       'num_of_atoms': 1,
       'residuals0': 0,
       'volume0': 7.8203795,
       'volume0_unit': 'A^3'}